In [2]:
exec(open('bool_hl.py').read())

In [3]:
n = 3
x_all3 = np.array(list(itertools.product(*[[0,1]]*n)))
y_all3 = np.array(list(itertools.product(*[[0,1]]*(2**n)))).T
u = 1 * (np.random.random(8) >= 0.5)
make_light_model(u)
light_model.summary()
light_model.evaluate(x_all3, u.reshape(8,1))


Model: "light_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3)]               0         
                                                                 
 dense (Dense)               (None, 7)                 28        
                                                                 
 dense_1 (Dense)             (None, 1)                 8         
                                                                 
Total params: 36 (144.00 Byte)
Trainable params: 36 (144.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
1/1 [==============================] - 0s 180ms/step - loss: 0.0000e+00 - accuracy: 1.0000


[0.0, 1.0]

In [4]:
def make_approx_light_model(u, m=None, d=1, r=0.001):
    global approx_light_model
    n = int(np.log2(u.shape[0]))
    if m is None:
        m = 2**n-1
    layers = [tf.keras.Input(shape=(n))]
    for i in range(d):
        layers.append(tf.keras.layers.Dense(m, activation='relu')(layers[-1]))
    out = layers.append(tf.keras.layers.Dense(1, activation='linear')(layers[-1]))
    approx_light_model = tf.keras.Model(inputs=layers[0], outputs=layers[-1], name='approx_light_model')
    opt = tf.keras.optimizers.Adam(learning_rate=r)
    approx_light_model.compile(loss="mean_squared_error", optimizer=opt, metrics=["mean_squared_error"])

def train_approx_light_model(u, epochs):
    n = int(np.log2(u.shape[0]))
    x_all = np.array(list(itertools.product(*[[0,1]]*n)))
    y_all = u.reshape(2**n,1)
    print(f'Shapes: x_all={x_all.shape}, y_all={y_all.shape}')
    ds_all = tf.data.Dataset.from_tensors((x_all, y_all))
    approx_light_model.fit(ds_all, epochs=epochs, verbose=0)

In [72]:
u = 1 * (np.random.random(8) >= 0.5)
# u = np.array([0,1,1,0,1,0,0,1])
# u = np.array([0,0,0,1,1,1,1,0])
print(u)
make_approx_light_model(u, m=3, d=1, r=0.02)
# approx_light_model.summary()
train_approx_light_model(u, 25000)
approx_light_model.evaluate(x_all3, u)
v = approx_light_model.predict(x_all3).reshape(8)
w = 1 * (v >= 0.5)
print(np.array([u,w]))
w0 = approx_light_model.layers[1].get_weights()[0]
b0 = approx_light_model.layers[1].get_weights()[1]
w1 = approx_light_model.layers[2].get_weights()[0]
b1 = approx_light_model.layers[2].get_weights()[1]
ww1 = np.round(w1 / np.abs(w1))
bb0 = np.round((b0 * np.abs(w1).T).reshape(3))
ww0 = np.round(w0 * np.abs(w1.T))
bb1 = np.round(b1)
approx_light_model.layers[2].set_weights([ww1, bb1])
approx_light_model.layers[1].set_weights([ww0, bb0])
approx_light_model.evaluate(x_all3, u)


[0 0 0 1 1 0 0 1]
Shapes: x_all=(8, 3), y_all=(8, 1)
1/1 [==============================] - 0s 34ms/step
[[0 0 0 1 1 0 0 1]
 [0 0 0 1 0 0 0 0]]
1/1 [==============================] - 0s 17ms/step - loss: 0.2500 - mean_squared_error: 0.2500


[0.25, 0.25]

In [67]:
print(np.round(ww0))
print(np.round(bb0))
print(np.round(ww1))
print(np.round(b1))

[[-0.  1.  1.]
 [ 1.  1.  0.]
 [ 1.  1. -1.]]
[-0. -1. -0.]
[[ 1.]
 [-1.]
 [ 1.]]
[0.]


In [63]:
approx_light_model.layers[2].set_weights([np.round(ww1), np.round(b1)])
approx_light_model.layers[1].set_weights([np.round(ww0), np.round(bb0)])
approx_light_model.evaluate(x_all3, u)


1/1 [==============================] - 0s 19ms/step - loss: 0.0000e+00 - mean_squared_error: 0.0000e+00


[0.0, 0.0]

In [13]:
print(w0 := approx_light_model.layers[1].get_weights()[0])
print(b0 := approx_light_model.layers[1].get_weights()[1])
print(w1 := approx_light_model.layers[2].get_weights()[0])
print(b1 := approx_light_model.layers[2].get_weights()[1])


[[-1.0900205   0.8072575  -1.3302488 ]
 [-0.64293903 -0.9164408   1.1472802 ]
 [-0.89299357  1.079913    0.1365029 ]]
[ 0.806028   -1.0909777  -0.60915065]
[[1.2406517]
 [1.255977 ]
 [1.4822887]]
[8.484423e-37]


In [47]:
D = np.diag(w1.reshape(3))
print(w0 @ D)
print(w0 * w1.T)

[[-1.3523358   1.0138968  -1.9718128 ]
 [-0.7976634  -1.1510286   1.7006005 ]
 [-1.1078941   1.356346    0.20233671]]
[[-1.3523358   1.0138968  -1.9718128 ]
 [-0.7976634  -1.1510286   1.7006005 ]
 [-1.1078941   1.356346    0.20233671]]


In [48]:
ww1 = w1 / np.abs(w1)
bb0 = (b0 * np.abs(w1).T).reshape(3)
ww0 = w0 * np.abs(w1.T)

In [39]:
ww0.shape, w0.shape

((3, 3), (3, 3))

In [49]:
t0 = np.array([1,2,3])
print(approx_light_model.predict(t0.reshape(1,3)))
print(w1.T @ np.maximum(w0.T @ t0 + b0,0) + b1)
print(ww1.T @ np.maximum(ww0.T @ t0 + bb0,0) + b1)

1/1 [==============================] - 0s 16ms/step
[[2.5440958]]
[2.54409588]
[2.54409583]


In [50]:
print(ww0)
print(bb0)
print(ww1)
print(b1)

[[-1.3523358   1.0138968  -1.9718128 ]
 [-0.7976634  -1.1510286   1.7006005 ]
 [-1.1078941   1.356346    0.20233671]]
[ 1.        -1.370243  -0.9029371]
[[1.]
 [1.]
 [1.]]
[8.484423e-37]


In [62]:
approx_light_model.layers[1].set_weights([np.array([[1,-1,-1],[-1,1,-1],[-1,-1,1],[1,1,1]]).T, np.array([0,0,0,-2])])
approx_light_model.layers[2].set_weights([np.array([[1,1,1,1]]).T, np.array([0])])
#approx_light_model.predict(x_all3)
v = approx_light_model.predict(x_all3).reshape(8)
w = 1 * (v >= 0.5)
print(np.array([u,w]))


1/1 [==============================] - 0s 15ms/step
[[0 1 1 0 1 0 0 1]
 [0 1 1 0 1 0 0 1]]


In [54]:
approx_light_model.layers[2].weights


[<tf.Variable 'dense_66/kernel:0' shape=(4, 1) dtype=float32, numpy=
 array([[ 0.3391559],
        [-0.7351508],
        [ 1.4711301],
        [-1.0265578]], dtype=float32)>,
 <tf.Variable 'dense_66/bias:0' shape=(1,) dtype=float32, numpy=array([0.49995717], dtype=float32)>]

In [5]:
metric_names = list([m.name for m in exact_model.metrics])
metrics = exact_model.evaluate(x_all, y_all)
[print(f'{u[0]}={u[1]}') for u in zip(metric_names, metrics)]
None

1/1 [==============================] - 0s 22ms/step - loss: 0.0000e+00 - accuracy: 1.0000
loss=0.0
accuracy=1.0


In [6]:
[print(f'{u[0]}={u[1]}') for u in list(zip([m.name for m in exact_model.metrics], exact_model.evaluate(x_all, y_all)))]

1/1 [==============================] - 0s 22ms/step - loss: 0.0000e+00 - accuracy: 1.0000
loss=0.0
accuracy=1.0


[None, None]

In [10]:
make_n_model(3)
n_model.summary()
# n_model.evaluate(x_all, y_all)

0out of8
1out of8
2out of8
3out of8
4out of8
5out of8
6out of8
7out of8
Model: "n_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 3)]               0         
                                                                 
 dense_4 (Dense)             (None, 7)                 28        
                                                                 
 dense_5 (Dense)             (None, 256)               2048      
                                                                 
Total params: 2076 (8.11 KB)
Trainable params: 2076 (8.11 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [21]:
make_n_model(3)
n_model.summary()

0out of8
1out of8
2out of8
3out of8
4out of8
5out of8
6out of8
7out of8
Model: "n_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 3)]               0         
                                                                 
 dense_10 (Dense)            (None, 7)                 28        
                                                                 
 dense_11 (Dense)            (None, 256)               2048      
                                                                 
Total params: 2076 (8.11 KB)
Trainable params: 2076 (8.11 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [11]:
n = 3
x_all3 = np.array(list(itertools.product(*[[0,1]]*n)))
y_all3 = np.array(list(itertools.product(*[[0,1]]*(2**n)))).T


In [12]:
n_model.evaluate(x_all3, y_all3)


1/1 [==============================] - 0s 146ms/step - loss: 0.0000e+00 - accuracy: 1.0000


[0.0, 1.0]

In [13]:
print(x_all3.shape)
print(y_all3.shape)

(8, 3)
(8, 256)


In [42]:
u = 1 * (np.random.random(8) >= 0.5)
make_light_model(u)
light_model.summary()
light_model.evaluate(x_all3, u.reshape(8,1))


Model: "light_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 3)]               0         
                                                                 
 dense_26 (Dense)            (None, 7)                 28        
                                                                 
 dense_27 (Dense)            (None, 1)                 8         
                                                                 
Total params: 36 (144.00 Byte)
Trainable params: 36 (144.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


1/1 [==============================] - 0s 137ms/step - loss: 0.0000e+00 - accuracy: 1.0000


[0.0, 1.0]

AttributeError: 'NoneType' object has no attribute 'compile'

In [33]:
u.shape

(8,)

In [10]:
make_approx_model()

In [11]:
train_approx_model(10)

Epoch 1/10


ValueError: in user code:

    File "c:\Program Files\Python311\Lib\site-packages\keras\src\engine\training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "c:\Program Files\Python311\Lib\site-packages\keras\src\engine\training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Program Files\Python311\Lib\site-packages\keras\src\engine\training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "c:\Program Files\Python311\Lib\site-packages\keras\src\engine\training.py", line 1080, in train_step
        y_pred = self(x, training=True)
    File "c:\Program Files\Python311\Lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Program Files\Python311\Lib\site-packages\keras\src\engine\input_spec.py", line 253, in assert_input_compatibility
        raise V